In [21]:
import sqlite3
import pandas as pd
import os
pd.set_option('display.max_columns', None)

In [22]:
conn = sqlite3.connect("./dbToAdd.db")
df = pd.read_sql_query("SELECT * FROM match", conn)

# andere manier, probleem hier is dat het de kolomnamen niet meeneemt:
""" c = conn.cursor()
c.execute('SELECT * FROM match')
data = c.fetchall()
conn.close()
# Convert the data to a pandas DataFrame
df = pd.DataFrame(data) """

df.head()


,_id,_time_c,_time_u,location,date,type,competition,face,distance,distance_units,...,meteo_winddir,notes,unrecorded_volume,unrecorded_score,time_prep,time_shoot,time_warn,forced_end,archived,_aid
0,1,1649573710,1649573710,lowlands almere,1649548800,1,1,10,70.0,0,...,NaN,None,None,None,None,None,None,None,0,8taWcn7KldKx
1,2,1649581541,1649581541,lowlands almere score 2,1649548800,1,1,10,70.0,0,...,NaN,None,None,None,None,None,None,None,0,8taWcn7KldKx
2,3,1650011153,1650040560,training ksm,1649980800,1,2,10,70.0,0,...,NaN,pees oude fast flight geel,None,None,None,None,None,None,0,8taWcn7KldKx
3,4,1650040466,1650040466,training ksm,1649980800,1,2,10,70.0,0,...,NaN,pees spectra 18 draads,None,None,None,None,None,None,0,8taWcn7KldKx
4,5,1650043914,1650043914,training ksm,1649980800,1,2,10,70.0,0,...,NaN,tiller 3mm,None,None,None,None,None,None,0,8taWcn7KldKx


In [23]:
# drop distance_units, sightadviceaccept_shot_id, time_prep, time_shoot, time_warn, forced_end
df = df.drop(columns=["distance_units", "sightadviceaccept_shot_id", "time_prep", "time_shoot", "time_warn", "forced_end"])
df.head()

,_id,_time_c,_time_u,location,date,type,competition,face,distance,maxshotsinserie,maxseries,bowsetup_id,arrowset_id,meteo_weather,meteo_wind,meteo_winddir,notes,unrecorded_volume,unrecorded_score,archived,_aid
0,1,1649573710,1649573710,lowlands almere,1649548800,1,1,10,70.0,6,6,1,1,NaN,2.0,NaN,None,None,None,0,8taWcn7KldKx
1,2,1649581541,1649581541,lowlands almere score 2,1649548800,1,1,10,70.0,6,6,1,1,2.0,3.0,NaN,None,None,None,0,8taWcn7KldKx
2,3,1650011153,1650040560,training ksm,1649980800,1,2,10,70.0,9,4,1,1,3.0,NaN,NaN,pees oude fast flight geel,None,None,0,8taWcn7KldKx
3,4,1650040466,1650040466,training ksm,1649980800,1,2,10,70.0,9,4,1,1,2.0,NaN,NaN,pees spectra 18 draads,None,None,0,8taWcn7KldKx
4,5,1650043914,1650043914,training ksm,1649980800,1,2,10,70.0,6,6,1,1,NaN,NaN,NaN,tiller 3mm,None,None,0,8taWcn7KldKx


In [25]:
df.isnull().sum()

_id                    0
_time_c                0
_time_u                0
location               0
date                   0
type                   0
competition            0
face                   0
distance               0
maxshotsinserie        0
maxseries              0
bowsetup_id            0
arrowset_id            0
meteo_weather        215
meteo_wind           284
meteo_winddir        290
notes                244
unrecorded_volume     69
unrecorded_score     295
archived               0
_aid                   0
dtype: int64

In [ ]:
# _time_c, _time_u, date parsen vanuit unix epoch
df['_time_c'] = pd.to_datetime(df['_time_c'], unit='s')
df['_time_u'] = pd.to_datetime(df['_time_u'], unit='s')
df['date'] = pd.to_datetime(df['date'], unit='s') 

# distance parsen naar int
#df['distance'] = df['distance'].astype(int) #use floats instead of ints to allow for NaN values

df.head()

In [29]:
# type, competition, face, meteo_weather, meteo_wind, decoden 
# Mapping dictionaries
type_mapping = {
    1: 'WA Outdoor',
    3: 'WA Indoor',
    0: 'Custom',
    12: 'FITA Outdoor'
}
competition_mapping = {
    0: 'training',
    1: 'competition',
    2: 'tuning'
}
face_mapping = {
    0: 'blank bale',
    10: '122cm',
    2: ''
}
metheo_weather_mapping = {
    0: 'sunny',
    1: 'cloudy with sun',
    2: 'cloudy',
    3: 'light rain',
    4: 'heavy rain',
    5: 'indoor',
}
meteo_wind_mapping = {
    0: '0bft',
    1: '1bft',
    2: '2bft',
    3: '3bft',
    4: '4bft',
    5: '5bft',
    6: '6bft',
    7: '7bft',
    8: '8bft',
    9: '9bft'
}
# Replace numerical values with string labels
df['type'] = df['type'].replace(type_mapping)
df['competition'] = df['competition'].replace(competition_mapping)
df['face'] = df['face'].replace(face_mapping)
df['meteo_weather'] = df['meteo_weather'].replace(metheo_weather_mapping)
df['meteo_wind'] = df['meteo_wind'].replace(meteo_wind_mapping)
df.head()

,_id,_time_c,_time_u,location,date,type,competition,face,distance,maxshotsinserie,maxseries,bowsetup_id,arrowset_id,meteo_weather,meteo_wind,meteo_winddir,notes,unrecorded_volume,unrecorded_score,archived,_aid
0,1,2022-04-10 06:55:10,2022-04-10 06:55:10,lowlands almere,2022-04-10,WA Outdoor,competition,122cm,70.0,6,6,1,1,NaN,2bft,NaN,None,None,None,0,8taWcn7KldKx
1,2,2022-04-10 09:05:41,2022-04-10 09:05:41,lowlands almere score 2,2022-04-10,WA Outdoor,competition,122cm,70.0,6,6,1,1,cloudy,3bft,NaN,None,None,None,0,8taWcn7KldKx
2,3,2022-04-15 08:25:53,2022-04-15 16:36:00,training ksm,2022-04-15,WA Outdoor,tuning,122cm,70.0,9,4,1,1,light rain,NaN,NaN,pees oude fast flight geel,None,None,0,8taWcn7KldKx
3,4,2022-04-15 16:34:26,2022-04-15 16:34:26,training ksm,2022-04-15,WA Outdoor,tuning,122cm,70.0,9,4,1,1,cloudy,NaN,NaN,pees spectra 18 draads,None,None,0,8taWcn7KldKx
4,5,2022-04-15 17:31:54,2022-04-15 17:31:54,training ksm,2022-04-15,WA Outdoor,tuning,122cm,70.0,6,6,1,1,NaN,NaN,NaN,tiller 3mm,None,None,0,8taWcn7KldKx
